In [19]:
!pip3 install git+https://github.com/informagi/REL

  Cloning https://github.com/informagi/REL to /private/var/folders/gl/m_rj4zy92d7dn3h0h0f4byh40000gn/T/pip-req-build-ppl_n25g
  Running command git clone -q https://github.com/informagi/REL /private/var/folders/gl/m_rj4zy92d7dn3h0h0f4byh40000gn/T/pip-req-build-ppl_n25g


  Created wheel for REL: filename=REL-0.0.1-py3-none-any.whl size=80532 sha256=e977ff3951ed5ebb50b50c37e9e06f39e48e2982dc3bcad1eea5eefe77274be8
  Stored in directory: /private/var/folders/gl/m_rj4zy92d7dn3h0h0f4byh40000gn/T/pip-ephem-wheel-cache-kjdiaw13/wheels/49/71/1c/17fc9ec22ba59dd3134ba094e90fee7921f7d467fa0293e5e3
Successfully built REL


In [21]:
from REL.mention_detection import MentionDetection
from REL.utils import process_results
from REL.entity_disambiguation import EntityDisambiguation
from REL.ner import Cmns, load_flair_ner

base_url = "/Users/iain/LocalStorage/coding/github/REL/"
wiki_year = '2014'
wiki_version = "wiki_" + wiki_year
model_path = "/Users/iain/LocalStorage/coding/github/REL/ed-wiki-{}/model".format(wiki_year)

def example_preprocessing():
    # user does some stuff, which results in the format below.
    text_1 = "Roger Federer (German pronunciation: [ˈrɔdʒər ˈfeːdərər]; born 8 August 1981) is a Swiss professional tennis player who is ranked world No. 4 in men's singles tennis by the Association of Tennis Professionals (ATP).[3] He has won 20 Grand Slam singles titles—the most in history for a male player—and has held the world No. 1 spot in the ATP rankings for a record total of 310 weeks (including a record 237 consecutive weeks) and was the year-end No. 1 five times, including four consecutive. Federer, who turned professional in 1998, was continuously ranked in the top 10 from October 2002 to November 2016."
    text_2 = "Federer has won a record eight Wimbledon men's singles titles, six Australian Open titles, five US Open titles (all consecutive, a record), and one French Open title. He is one of eight men to have achieved a Career Grand Slam. Federer has reached a record 31 men's singles Grand Slam finals, including 10 consecutively from the 2005 Wimbledon Championships to the 2007 US Open. Federer has also won a record six ATP Finals titles, 28 ATP Tour Masters 1000 titles, and a record 24 ATP Tour 500 titles. Federer was a member of Switzerland's winning Davis Cup team in 2014. He is also the only player after Jimmy Connors to have won 100 or more career singles titles, as well as to amass 1,200 wins in the Open Era."

    processed = {"test_doc1": [text_1, []], "test_doc2": [text_2, []]}
    return processed


input_text = example_preprocessing()

mention_detection = MentionDetection(base_url, wiki_version)
tagger_ner = load_flair_ner("ner-fast")
#tagger_ngram = Cmns(base_url, wiki_version, n=5)
mentions_dataset, n_mentions = mention_detection.find_mentions(input_text, tagger_ner)

config = {
    "mode": "eval",
    "model_path": model_path,
}

model = EntityDisambiguation(base_url, wiki_version, config)
predictions, timing = model.predict(mentions_dataset)

result = process_results(mentions_dataset, predictions, input_text)
print('*** result ***')
print(result)

2020-07-14 12:12:10,024 loading file /Users/iain/.flair/models/en-ner-fast-conll03-v0.4.pt


FileNotFoundError: [Errno 2] No such file or directory: '/Users/iain/LocalStorage/virtualenvs/multi_task_merge_env/lib/python3.7/site-packages/REL/models/models.json'

In [18]:
import requests
import time

IP_ADDRESS = "https://rel.cs.ru.nl/api"
text_doc = "If you're going to try, go all the way - Charles Bukowski"

text_1 = "Roger Federer (German pronunciation: [ˈrɔdʒər ˈfeːdərər]; born 8 August 1981) is a Swiss professional tennis player who is ranked world No. 4 in men's singles tennis by the Association of Tennis Professionals (ATP).[3] He has won 20 Grand Slam singles titles—the most in history for a male player—and has held the world No. 1 spot in the ATP rankings for a record total of 310 weeks (including a record 237 consecutive weeks) and was the year-end No. 1 five times, including four consecutive. Federer, who turned professional in 1998, was continuously ranked in the top 10 from October 2002 to November 2016."
text_2 = "Federer has won a record eight Wimbledon men's singles titles, six Australian Open titles, five US Open titles (all consecutive, a record), and one French Open title. He is one of eight men to have achieved a Career Grand Slam. Federer has reached a record 31 men's singles Grand Slam finals, including 10 consecutively from the 2005 Wimbledon Championships to the 2007 US Open. Federer has also won a record six ATP Finals titles, 28 ATP Tour Masters 1000 titles, and a record 24 ATP Tour 500 titles. Federer was a member of Switzerland's winning Davis Cup team in 2014. He is also the only player after Jimmy Connors to have won 100 or more career singles titles, as well as to amass 1,200 wins in the Open Era."


start = time.time()
for i in range(5):
    for text in [text_1, text_2]:
        # Example ED.
        document = {
            "text": text,
            "spans": []
        }
        API_result = requests.post("{}".format(IP_ADDRESS), json=document).json()
        print(API_result)
print(time.time()-start)

[[0, 13, 'Roger Federer', 'Roger_Federer', 0.9943608641624451, 'PER', 0.0], [15, 6, 'German', 'Germany', 0.9972993731498718, 'MISC', 0.0], [83, 5, 'Swiss', 'Switzerland', 0.9969614148139954, 'MISC', 0.0], [173, 35, 'Association of Tennis Professionals', 'Association_of_Tennis_Professionals', 0.7372175604104996, 'ORG', 0.0], [210, 3, 'ATP', 'Association_of_Tennis_Professionals', 0.8938578963279724, 'ORG', 0.0], [233, 10, 'Grand Slam', 'Grand_Slam_(tennis)', 0.9843668639659882, 'MISC', 0.0], [338, 3, 'ATP', 'Association_of_Tennis_Professionals', 0.6155640482902527, 'ORG', 0.0], [493, 7, 'Federer', 'Roger_Federer', 0.9980781078338623, 'PER', 0.0]]
[[0, 7, 'Federer', 'Roger_Federer', 0.9957826137542725, 'PER', 0.0], [31, 9, 'Wimbledon', 'The_Championships,_Wimbledon', 0.9898585677146912, 'MISC', 0.0], [67, 15, 'Australian Open', 'Australian_Open', 0.9834259152412415, 'MISC', 0.0], [96, 7, 'US Open', 'US_Open_(tennis)', 0.9788042902946472, 'MISC', 0.0], [148, 11, 'French Open', 'French_Open

[[0, 7, 'Federer', 'Roger_Federer', 0.9957826137542725, 'PER', 0.0], [31, 9, 'Wimbledon', 'The_Championships,_Wimbledon', 0.9898585677146912, 'MISC', 0.0], [67, 15, 'Australian Open', 'Australian_Open', 0.9834259152412415, 'MISC', 0.0], [96, 7, 'US Open', 'US_Open_(tennis)', 0.9788042902946472, 'MISC', 0.0], [148, 11, 'French Open', 'French_Open', 0.8920957446098328, 'MISC', 0.0], [209, 17, 'Career Grand Slam', 'Grand_Slam_(tennis)', 0.7972376545270284, 'MISC', 0.0], [228, 7, 'Federer', 'Roger_Federer', 0.9952180981636047, 'PER', 0.0], [274, 10, 'Grand Slam', 'Grand_Slam_(tennis)', 0.9383337199687958, 'MISC', 0.0], [334, 9, 'Wimbledon', 'The_Championships,_Wimbledon', 0.7505090236663818, 'MISC', 0.0], [370, 7, 'US Open', 'US_Open_(tennis)', 0.9500445425510406, 'MISC', 0.0], [379, 7, 'Federer', 'Roger_Federer', 0.9968993663787842, 'PER', 0.0], [481, 8, 'ATP Tour', 'Association_of_Tennis_Professionals', 0.8084564805030823, 'MISC', 0.0], [502, 7, 'Federer', 'Roger_Federer', 0.995793581008